In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import the necessary modules
import pandas as pd
import math
import random
import matplotlib.pyplot as plt
from scipy import stats
import sys
import os
import numpy as np
import seaborn as sns

#Load Dataset

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/MRP/Current/Deliverables/Methodology_&_Experiments/Report/Coding/Data_files/DataCoSupplyChainDataset.csv', encoding='latin-1')

In [ ]:
# preprocessing.py

import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose

def preprocess_data(file_path):
    df = pd.read_csv(file_path, encoding='latin1')
    df.drop_duplicates(inplace=True)
    df.columns = df.columns.str.strip()

    columns_to_drop = ['Product Description', 'Product Image', 'Order Zipcode', 'Customer Email', 'Customer Password']
    df.drop(columns=columns_to_drop, errors='ignore', inplace=True)

    df = df.dropna(subset=['Customer Zipcode', 'Customer Lname'])

    df['order date (DateOrders)'] = pd.to_datetime(df['order date (DateOrders)'])
    df['shipping date (DateOrders)'] = pd.to_datetime(df['shipping date (DateOrders)'])

    df['order_date'] = df['order date (DateOrders)'].dt.date
    df['order_time'] = df['order date (DateOrders)'].dt.time
    df['order_hour'] = df['order date (DateOrders)'].dt.hour
    df['order_week'] = df['order date (DateOrders)'].dt.isocalendar().week
    df['order_month'] = df['order date (DateOrders)'].dt.month
    df['order_dayofweek'] = df['order date (DateOrders)'].dt.dayofweek
    df['order_weekend'] = df['order date (DateOrders)'].dt.dayofweek >= 5

    df['shipping_date'] = df['shipping date (DateOrders)'].dt.date
    df['shipping_time'] = df['shipping date (DateOrders)'].dt.time
    df['shipping_hour'] = df['shipping date (DateOrders)'].dt.hour
    df['shipping_week'] = df['shipping date (DateOrders)'].dt.isocalendar().week
    df['shipping_month'] = df['shipping date (DateOrders)'].dt.month
    df['shipping_dayofweek'] = df['shipping date (DateOrders)'].dt.dayofweek
    df['shipping_weekend'] = df['shipping date (DateOrders)'].dt.dayofweek >= 5

    df = df.set_index('order date (DateOrders)')

    monthly_sales = df['Sales'].resample('M').sum()
    decomposition = seasonal_decompose(monthly_sales, model='additive', period=7)

    plt.figure(figsize=(12, 8))
    plt.subplot(411); plt.plot(monthly_sales); plt.title('Original')
    plt.subplot(412); plt.plot(decomposition.trend); plt.title('Trend')
    plt.subplot(413); plt.plot(decomposition.seasonal); plt.title('Seasonality')
    plt.subplot(414); plt.plot(decomposition.resid); plt.title('Residual')
    plt.tight_layout(); plt.savefig("seasonal_decomposition.png"); plt.close()

    return df


In [ ]:
df = preprocess_data('/content/drive/MyDrive/MRP/Current/Deliverables/Methodology_&_Experiments/Report/Coding/Data_files/DataCoSupplyChainDataset.csv')

/tmp/ipython-input-4-868762057.py:41: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_sales = df['Sales'].resample('M').sum()


In [ ]:
# rename the index col to date
df = df.rename_axis('date')

In [ ]:
display(df)

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,order_month,order_dayofweek,order_weekend,shipping_date,shipping_time,shipping_hour,shipping_week,shipping_month,shipping_dayofweek,shipping_weekend
date,,,,,,,,,,,,,,,,,,,,,
2018-01-31 22:56:00,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,1,2,False,2018-02-03,22:56:00,22,5,2,5,True
2018-01-13 12:27:00,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,1,5,True,2018-01-18,12:27:00,12,3,1,3,False
2018-01-13 12:06:00,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,1,5,True,2018-01-17,12:06:00,12,3,1,2,False
2018-01-13 11:45:00,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,1,5,True,2018-01-16,11:45:00,11,3,1,1,False
2018-01-13 11:24:00,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,1,5,True,2018-01-15,11:24:00,11,3,1,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-01-16 03:40:00,CASH,4,4,40.000000,399.980011,Shipping on time,0,45,Fishing,Brooklyn,...,1,5,True,2016-01-20,03:40:00,3,3,1,2,False
2016-01-16 01:34:00,DEBIT,3,2,-613.770019,395.980011,Late delivery,1,45,Fishing,Bakersfield,...,1,5,True,2016-01-19,01:34:00,1,3,1,1,False
2016-01-15 21:00:00,TRANSFER,5,4,141.110001,391.980011,Late delivery,1,45,Fishing,Bristol,...,1,4,False,2016-01-20,21:00:00,21,3,1,2,False


In [ ]:
# Save as Data_Co.csv
df.to_csv('/content/drive/MyDrive/MRP/Current/Deliverables/Methodology_&_Experiments/Report/Coding/Data_files/Data_Co.csv')